# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [83]:
from sklearn import cross_validation, datasets, metrics, tree, model_selection
from sklearn.cross_validation import train_test_split
import numpy as np

X, y  = datasets.load_boston(return_X_y=True)

#(X_train, X_test) = train_test_split(X, test_size=0.25, random_state=0)
#(y_train, y_test) = train_test_split(y, test_size=0.25, random_state=0)

n_split = np.int(np.round(y.shape[0]*0.75))

X_train = X[:n_split]

y_train = y[:n_split]

X_test = X[n_split:]

y_test = y[n_split:]

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [84]:
def L_derivative(y_train, z):
    return y_train - z

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [85]:
import sklearn
# Функция из задания
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

# Списки для базовых алгоритмов и коэффициентов
base_algorithms_list = []
coefficients_list = [0.9]

# Обучение первого алгоритм
b = sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42)
b.fit(X_train, y_train)
#Вычисление коэффициента z и добавление первого алгоритма в список базовых алгоритмов
z = gbm_predict(X_train)
base_algorithms_list.append(b)
# Среднеквадратичная ошибка на основе первого базового алгоритма
print sklearn.metrics.mean_squared_error(y_test, gbm_predict(X_test))**0.5

# Циклическое создание и обучение алгоритмов с учетом смещения
# и добавление базовых алгоритмов и коэффициентов в соответствующие списки
for i in range(49):
    b = sklearn.tree.DecisionTreeRegressor(max_depth=5 , random_state=42)
    b.fit(X_train, L_derivative(y_train, gbm_predict(X_train)))
    base_algorithms_list.append(b)
    coefficients_list.append(0.9)
        
# Среднеквадратичная ошибка на основе обученного алгоритма
print sklearn.metrics.mean_squared_error(y_test, gbm_predict(X_test))**0.5

4.674470252481672
5.455472074527531


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [86]:
# Списки для базовых алгоритмов и коэффициентов
base_algorithms_list = []
coefficients_list = [0.9]
# Обучение первого алгоритм
b = sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42)
b.fit(X_train, y_train)
#Вычисление коэффициента z и добавление первого алгоритма в список базовых алгоритмов
z = gbm_predict(X_train)
base_algorithms_list.append(b)
# Среднеквадратичная ошибка на основе первого базового алгоритма
print sklearn.metrics.mean_squared_error(y_test, gbm_predict(X_test))**0.5

# Циклическое создание и обучение алгоритмов с учетом смещения
# и добавление базовых алгоритмов и коэффициентов в соответствующие списки
for i in range(49):
    b = sklearn.tree.DecisionTreeRegressor(max_depth=5 , random_state=42)
    b.fit(X_train, L_derivative(y_train, gbm_predict(X_train)))
    base_algorithms_list.append(b)
    coefficients_list.append(0.9 / (1.0 + i))
print sklearn.metrics.mean_squared_error(y_test, gbm_predict(X_test))**0.5

4.674470252481672
5.166369740268366


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [87]:
import xgboost as xgb
import time
# read in data
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
# specify parameters via map
for i in range(100):
    param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'tree_method': 'gpu_hist'}
    gpu_res = {}
    tmp = time.time()
    num_round = 2
    bst = xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_res)
    print("GPU Training Time: %s seconds" % (str(time.time() - tmp)))
    preds = bst.predict(dtest)
    print(preds)       
    
#param = {'objective': 'multi:softmax', # Specify multiclass classification
#         'num_class': 8, # Number of possible output classes
#         'tree_method': 'gpu_hist' # Use GPU accelerated algorithm
#         }

# Convert input data from numpy to XGBoost format
#dtrain = xgb.DMatrix(X_train, label=y_train)
#dtest = xgb.DMatrix(X_test, label=y_test)

#gpu_res = {} # Store accuracy result
#tmp = time.time()
# Train model
#xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_res)
#print("GPU Training Time: %s seconds" % (str(time.time() - tmp)))

XGBoostError: [20:11:03] C:/Users/Administrator/Desktop/xgboost/src/objective/regression_obj.cu:102: label must be in [0,1] for logistic regression

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [88]:
from sklearn import linear_model
reg = sklearn.linear_model.LinearRegression()
reg.fit(X_train, y_train)
print sklearn.metrics.mean_squared_error(y_test, reg.predict(X_test))**0.5

7.848121796479956
